###Import the required libraries

In [0]:

import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import layers
from tensorflow.keras import Model
import os
import zipfile


###Retrieve the required dataset

In [0]:

# Download the training set
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/rps.zip \
    -O /tmp/rps.zip
  
# Download the test set
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/rps-test-set.zip \
    -O /tmp/rps-test-set.zip

# Unzip the training set
local_zip = '/tmp/rps.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp/')
zip_ref.close()

# Unzip the test set
local_zip = '/tmp/rps-test-set.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp/')
zip_ref.close()

# Create the dataset directories
rock_dir = os.path.join('/tmp/rps/rock')
paper_dir = os.path.join('/tmp/rps/paper')
scissors_dir = os.path.join('/tmp/rps/scissors')

# Print the size of the dataset
print('total training rock images:', len(os.listdir(rock_dir)))
print('total training paper images:', len(os.listdir(paper_dir)))
print('total training scissors images:', len(os.listdir(scissors_dir)))


###Pre-process and augment the data

In [5]:

from keras_preprocessing.image import ImageDataGenerator

# Pre-process and augment the training set
TRAINING_DIR = "/tmp/rps/"
training_datagen = ImageDataGenerator(rescale = 1./255,
	                                    rotation_range = 40,
                                      width_shift_range = 0.2,
                                      height_shift_range = 0.2,
                                      shear_range = 0.2,
                                      zoom_range = 0.2,
                                      horizontal_flip = True,
                                      fill_mode = 'nearest')
train_generator = training_datagen.flow_from_directory(TRAINING_DIR,
                                                       target_size = (150,150),
                                                       class_mode = 'categorical',
                                                       batch_size = 20)

# Pre-process the test set
VALIDATION_DIR = "/tmp/rps-test-set/"
validation_datagen = ImageDataGenerator(rescale = 1./255)
validation_generator = validation_datagen.flow_from_directory(VALIDATION_DIR,
                                                              target_size = (150,150),
                                                              class_mode = 'categorical',
                                                              batch_size = 10)


Found 2520 images belonging to 3 classes.
Found 372 images belonging to 3 classes.


###Define the model

In [0]:

# Define the layers for the model
rps_model = tf.keras.models.Sequential ([
    tf.keras.layers.Conv2D(32, (3,3), activation = 'relu', input_shape = (150, 150, 3), name = 'Conv1'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D((2, 2), name = 'Pool1'),
    tf.keras.layers.Conv2D(64, (3,3), activation = 'relu', name = 'Conv2'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D((2,2), name = 'Pool2'),
    tf.keras.layers.Conv2D(128, (3,3), activation = 'relu', name = 'Conv3'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D((2,2), name = 'Pool3'),
    tf.keras.layers.Conv2D(128, (3,3), activation = 'relu', name = 'Conv4'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D((2,2), name = 'Pool4'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation = 'relu', name = 'Dense5'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(3, activation = 'softmax', name = 'Output6')
])

# Define the optimizer and loss function for the model
rps_model.compile(loss = 'categorical_crossentropy',
                  optimizer='rmsprop',
                  metrics=['accuracy'])


###Train the model

In [0]:

# Fit the model and store the intermediate outputs
history = rps_model.fit(train_generator,
                        epochs = 25, 
                        steps_per_epoch = 126, 
                        validation_data = validation_generator, 
                        validation_steps = 38,
                        verbose = 1)


###Plot the loss and accuracy of the model w.r.t the number of epochs

In [0]:

# Retrieve a list of results on training and test datasets for each training epoch
acc      = history.history['accuracy']
val_acc  = history.history['val_accuracy']
loss     = history.history['loss']
val_loss = history.history['val_loss']

# Get the total number of epochs
epochs   = range(len(acc))

# Plot training and validation accuracy per epoch
plt.plot  (epochs, acc)
plt.plot  (epochs, val_acc)
plt.title ('Training and validation accuracy', color = "white")
plt.figure()

# Plot training and validation loss per epoch
plt.plot  (epochs, loss)
plt.plot  (epochs, val_loss)
plt.title ('Training and validation loss', color = "white")


###Get a summary of the model

In [0]:

# Print the summary of the model
rps_model.summary()
